In [1]:
!pip install -q tensorflow_addons
!pip install -q pyarrow
!pip install -q wandb

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sn
import datetime

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, GroupShuffleSplit

import glob
import sys
import os
import math
import gc
import sys
import sklearn
import scipy
import io
import json

# import wandb

tf.get_logger().setLevel('INFO')

2023-04-19 06:18:40.818963: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 06:18:41.439375: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/tensorflow/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
# # Setup Weights and Biases
# wandb.login()

In [4]:
# LOG_DIR = './logs/fit'
# wandb.tensorboard.patch(root_logdir= LOG_DIR)
# wandb.init(project='w251-GISLR-Final', 
#            config=config,
#           sync_tensorboard=True)

# Config

In [5]:
config = {
    
    "N_COLS0": 543,
    "N_COLS": 227,
    "N_ROWS":543,
    "N_DIMS": 2, 
    "N_EPOCHS": 150,
    "TRAIN_BATCH_SIZE": 128,
    "INPUT_SIZE":38,
    "BATCH_ALL_SIGNS_N": 4,
    "NUM_CLASSES": 250,
    "BATCH_SIZE_VAL":128,
    "WD_RATIO":0.05,
    "LEARNING_RATE": 0.01,
    "WEIGHT_DECAY": 0.00001,
    "N_WARMUP_EPOCHS": 0,
    "MASK_VAL": 4237
}


In [6]:
# Source: https://www.kaggle.com/competitions/asl-signs/overview/evaluation
ROWS_PER_FRAME = 543  # number of landmarks per frame
#w251-asl-data/raw-data/train_landmark_files/28656/3311214787.parquet

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [7]:
# Prints Shape and Dtype For List Of Variables
def print_shape_dtype(l, names):
    for e, n in zip(l, names):
        print(f'{n} shape: {e.shape}, dtype: {e.dtype}')


In [8]:
import tensorflow as tf
# import numpy as np

class PreprocessLayerV2(tf.keras.layers.Layer):
    def __init__(self, INPUT_SIZE):
        super(PreprocessLayerV2, self).__init__()
        self.INPUT_SIZE = INPUT_SIZE
        # Indicies in original data. 
        self.FACE_IDXS = tf.constant([0, 6, 7, 11, 12, 13, 14, 15, 17, 22, 23, 24, 25, 26, 30, 31, 
                     33, 37, 38, 39, 40, 41, 42, 56, 61, 62, 72, 73, 74, 76, 77, 
                     78, 80, 81, 82, 84, 86, 87, 88, 89, 90, 91, 95, 96, 110, 112, 
                     113, 122, 128, 130, 133, 144, 145, 146, 153, 154, 155, 157, 158, 
                     159, 160, 161, 163, 168, 173, 178, 179, 180, 181, 183, 184, 185, 
                     188, 189, 190, 191, 193, 196, 197, 232, 233, 243, 244, 245, 246, 
                     247, 249, 252, 253, 254, 255, 256, 259, 260, 263, 267, 268, 269, 
                     270, 271, 272, 286, 291, 292, 302, 303, 304, 306, 307, 308, 310, 
                     311, 312, 314, 316, 317, 318, 319, 320, 321, 324, 325, 339, 341, 
                     351, 357, 359, 362, 373, 374, 375, 380, 381, 382, 384, 385, 386, 
                     387, 388, 390, 398, 402, 403, 404, 405, 407, 408, 409, 412, 413, 
                     414, 415, 417, 419, 453, 463, 464, 465, 466, 467], dtype=tf.int32)
        self.POSE_IDXS = tf.constant(tf.range(489, 514, delta=1, dtype=tf.int32))
        self.LEFT_HAND_IDXS = tf.constant(tf.range(468, 489, delta=1, dtype=tf.int32))
        self.RIGHT_HAND_IDXS = tf.constant(tf.range(522, 543, delta=1, dtype=tf.int32))
        
        self.HAND_IDXS = tf.constant(tf.concat([self.LEFT_HAND_IDXS, self.RIGHT_HAND_IDXS], 0), dtype=tf.int32)
            
        # All landmarks that are used for modeling. 
        self.LANDMARK_IDXS = tf.constant(tf.concat([self.FACE_IDXS, self.POSE_IDXS, self.LEFT_HAND_IDXS, self.RIGHT_HAND_IDXS], 0), dtype=tf.int32)
        
        # Indicies after landmarks have been filtered. 
        self.FACE_START = tf.constant(0, dtype=tf.int32)
        self.LEFT_HAND_START = tf.constant(len(self.FACE_IDXS), dtype=tf.int32)
        self.POSE_START = tf.constant(self.LEFT_HAND_START + len(self.LEFT_HAND_IDXS), dtype=tf.int32)
        self.RIGHT_HAND_START = tf.constant(self.POSE_START + len(self.POSE_IDXS), dtype=tf.int32)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'INPUT_SIZE': self.INPUT_SIZE,
        })
        return config
    
    
    @tf.function(input_signature=(tf.TensorSpec(shape=[None, 543, 2], dtype=tf.float32),),)
    def call(self, data):
        
        # Filter Out Frames With Empty Hand Data
        frames_hands_nansum = tf.experimental.numpy.nanmean(tf.gather(data, self.HAND_IDXS, axis=1), axis=[1,2])
        non_empty_frames_idxs = tf.where(frames_hands_nansum > 0)
        non_empty_frames_idxs = tf.squeeze(non_empty_frames_idxs, axis=1)
        data = tf.gather(data, non_empty_frames_idxs, axis=0)
        
        # Cast Indices in float32 to be compatible with Tensorflow Lite
        non_empty_frames_idxs = tf.cast(non_empty_frames_idxs, tf.float32) 
        
        N_FRAMES = tf.shape(data)[0]
        data = tf.gather(data, self.LANDMARK_IDXS, axis=1)
        
        # Slice out face indicies, normalize across batch.        
        face = tf.slice(data, [0, self.FACE_START, 0], [N_FRAMES, self.LEFT_HAND_START, 2])
        # face_mean, face_std = self.get_mean_std(self.FACE_IDXS, face)
        xs = tf.reshape(tf.transpose(face, [2,0,1]), [2,tf.size(self.FACE_IDXS)*tf.shape(face)[0]])[0]
        ys = tf.reshape(tf.transpose(face, [2,0,1]), [2,tf.size(self.FACE_IDXS)*tf.shape(face)[0]])[1]
            
        FACE_MEAN_X = tf.math.reduce_mean(xs)
        FACE_STD_X = tf.math.reduce_std(xs)
        FACE_MEAN_Y = tf.math.reduce_mean(ys)
        FACE_STD_Y = tf.math.reduce_std(ys)

        face_mean = tf.stack([FACE_MEAN_X, FACE_MEAN_Y])
        face_std = tf.stack([FACE_STD_X, FACE_STD_Y])
        face = tf.where(
                    tf.math.equal(face, 0.0),
                    0.0,
                    (face - face_mean) / face_std,
                )
        
#         # Slice out left_hand indicies, normalize across batch.
        left_hand = tf.slice(data, [0, self.LEFT_HAND_START, 0], [N_FRAMES, self.POSE_START-self.LEFT_HAND_START, 2])
        xs = tf.reshape(tf.transpose(left_hand, [2,0,1]), [2,tf.size(self.LEFT_HAND_IDXS)*tf.shape(left_hand)[0]])[0]
        ys = tf.reshape(tf.transpose(left_hand, [2,0,1]), [2,tf.size(self.LEFT_HAND_IDXS)*tf.shape(left_hand)[0]])[1]
            
        LEFT_HAND_MEAN_X = tf.math.reduce_mean(xs)
        LEFT_HAND_STD_X = tf.math.reduce_std(xs)
        LEFT_HAND_MEAN_Y = tf.math.reduce_mean(ys)
        LEFT_HAND_STD_Y = tf.math.reduce_std(ys)

        left_hand_mean = tf.stack([LEFT_HAND_MEAN_X, LEFT_HAND_MEAN_Y])
        left_hand_std = tf.stack([LEFT_HAND_STD_X, LEFT_HAND_STD_Y])
        left_hand = tf.where(
                    tf.math.equal(left_hand, 0.0),
                    0.0,
                    (left_hand - left_hand_mean) / left_hand_std,
                )
        
#         # Slice out pose indicies, normalize across batch.
        pose = tf.slice(data, [0, self.POSE_START, 0], [N_FRAMES, self.RIGHT_HAND_START-self.POSE_START, 2])
        xs = tf.reshape(tf.transpose(pose, [2,0,1]), [2,tf.size(self.POSE_IDXS)*tf.shape(pose)[0]])[0]
        ys = tf.reshape(tf.transpose(pose, [2,0,1]), [2,tf.size(self.POSE_IDXS)*tf.shape(pose)[0]])[1]
            
        POSE_MEAN_X = tf.math.reduce_mean(xs)
        POSE_STD_X = tf.math.reduce_std(xs)
        POSE_MEAN_Y = tf.math.reduce_mean(ys)
        POSE_STD_Y = tf.math.reduce_std(ys)

        pose_mean = tf.stack([POSE_MEAN_X, POSE_MEAN_Y])
        pose_std = tf.stack([POSE_STD_X, POSE_STD_Y])
        pose = tf.where(
                    tf.math.equal(pose, 0.0),
                    0.0,
                    (pose - pose_mean) / pose_std,
                )
        
         # Slice out right_hand indicies, normalize across batch.
        right_hand = tf.slice(data, [0, self.RIGHT_HAND_START, 0], [N_FRAMES, tf.shape(data)[1] - self.RIGHT_HAND_START, 2])
        xs = tf.reshape(tf.transpose(right_hand, [2,0,1]), [2,tf.size(self.LEFT_HAND_IDXS)*tf.shape(right_hand)[0]])[0]
        ys = tf.reshape(tf.transpose(right_hand, [2,0,1]), [2,tf.size(self.LEFT_HAND_IDXS)*tf.shape(right_hand)[0]])[1]
            
        RIGHT_HAND_MEAN_X = tf.math.reduce_mean(xs)
        RIGHT_HAND_STD_X = tf.math.reduce_std(xs)
        RIGHT_HAND_MEAN_Y = tf.math.reduce_mean(ys)
        RIGHT_HAND_STD_Y = tf.math.reduce_std(ys)

        right_hand_mean = tf.stack([RIGHT_HAND_MEAN_X, RIGHT_HAND_MEAN_Y])
        right_hand_std = tf.stack([RIGHT_HAND_STD_X, RIGHT_HAND_STD_Y])
        right_hand = tf.where(
                    tf.math.equal(right_hand, 0.0),
                    0.0,
                    (right_hand - right_hand_mean) / right_hand_std,
                )
        
        
        # Concat landmarks back into same frame.
        data = tf.concat([face, left_hand, pose, right_hand], 1)
        
        
        # Video fits in self.INPUT_SIZE
        if N_FRAMES < self.INPUT_SIZE: # Number of frames we want
            # Attention mask for frames that contain data. 
            
            non_empty_frames_idxs = tf.pad(non_empty_frames_idxs, [[0, self.INPUT_SIZE-N_FRAMES]], constant_values=-1)
            data = tf.pad(data, [[0, self.INPUT_SIZE-N_FRAMES], [0,0], [0,0]], constant_values=0)
            # Fill NaN Values With 0
            data = tf.where(tf.math.is_nan(data), 0.0, data)
            # Reshape into (Number of desired frames, (Number of landmarks * 2))
            data = tf.reshape(data, [self.INPUT_SIZE, tf.shape(data)[1] * 2])
            return data, non_empty_frames_idxs
        # Video needs to be downsampled to INPUT_SIZE
        else:
            # Downsample video using nearest interpolation method. 
            data = tf.image.resize(data, size=(self.INPUT_SIZE, data.shape[1]), method='nearest')
            # Fill NaN Values With 0
            data = tf.where(tf.math.is_nan(data), 0.0, data)
            # Reshape into (Number of desired frames, (Number of landmarks * 2)).
            data = tf.reshape(data, [self.INPUT_SIZE, tf.shape(data)[1] * 2])
            # Create attention mask with all frames. 
            non_empty_frames_idxs = tf.range(0, self.INPUT_SIZE, 1, dtype=tf.float32)
            return data, non_empty_frames_idxs

In [9]:
train_metadata = pd.read_csv('train.csv')

N_SAMPLES = len(train_metadata)
print(f'N_SAMPLES: {N_SAMPLES}')

N_SAMPLES: 94477


In [10]:
# Get complete file path to file
def get_file_path(path):
    return f'/kaggle/input/asl-signs/{path}'

train_metadata['file_path'] = train_metadata['path'].apply(get_file_path)

In [11]:
# Add ordinally Encoded Sign (assign number to each sign name)
train_metadata['sign_ord'] = train_metadata['sign'].astype('category').cat.codes

# Dictionaries to translate sign <-> ordinal encoded sign
SIGN2ORD = train_metadata[['sign', 'sign_ord']].set_index('sign').squeeze().to_dict()
ORD2SIGN = train_metadata[['sign_ord', 'sign']].set_index('sign_ord').squeeze().to_dict()

In [12]:
preprocess_layer = PreprocessLayerV2(config["INPUT_SIZE"])

2023-04-19 06:18:43.210161: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-19 06:18:43.230003: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-19 06:18:43.232092: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [13]:
"""
    face: 0:468
    left_hand: 468:489
    pose: 489:522
    right_hand: 522:544
        
"""
def get_data(file_path):
    # Load Raw Data
    data = load_relevant_data_subset(file_path)
    # Process Data Using Tensorflow
    data = preprocess_layer(data)
    
    return data

In [14]:
# Get the full dataset
def preprocess_dataset():
    # Create arrays to save data
    X = np.zeros([N_SAMPLES, config["INPUT_SIZE"], config["N_COLS"] * config["N_DIMS"]], dtype=np.float32)
    y = np.zeros([N_SAMPLES], dtype=np.int32)
    NON_EMPTY_FRAME_IDXS = np.full([N_SAMPLES, config["INPUT_SIZE"]], -1, dtype=np.float32)

    for row_idx, (file_path, sign_ord) in enumerate(tqdm(train_metadata[['file_path', 'sign_ord']].values)):
        if row_idx % 5000 == 0:
            print(f'Generated {row_idx}/{N_SAMPLES}')

        data, non_empty_frame_idxs = get_data(file_path)
        X[row_idx] = data
        y[row_idx] = sign_ord
        NON_EMPTY_FRAME_IDXS[row_idx] = non_empty_frame_idxs
        if np.isnan(data).sum() > 0:
            print(row_idx)
            return data
    
    # Save X/y
    np.save('X.npy', X)
    np.save('y.npy', y)
    np.save('NON_EMPTY_FRAME_IDXS.npy', NON_EMPTY_FRAME_IDXS)
        
    return X, y, NON_EMPTY_FRAME_IDXS

In [15]:
# X, y, NON_EMPTY_FRAME_IDXS = preprocess_dataset()

# print_shape_dtype([X, y, NON_EMPTY_FRAME_IDXS], ['X', 'y', 'NON_EMPTY_FRAME_IDXS'])
# print(f'# NaN Values X: {np.isnan(X).sum()}')

In [16]:
USE_VAL = True

DIM_NAMES = ['x', 'y']
SEED = 42

TRANSFORMERV1 = True

# Epsilon value for layer normalisation
LAYER_NORM_EPS = 1e-6

# Dense layer units for landmarks
FACE_UNITS = 384
HANDS_UNITS = 384
POSE_UNITS = 384
# final embedding and transformer embedding size
UNITS = 512

# Transformer
NUM_BLOCKS = 4
MLP_RATIO = 2
NUM_HEADS = 4

# Dropout
EMBEDDING_DROPOUT = 0.0
MLP_DROPOUT_RATIO = 0.1
CLASSIFIER_DROPOUT_RATIO = 0.1

# Initiailizers
# INIT_HE_UNIFORM = tf.keras.initializers.he_uniform
INIT_GLOROT_UNIFORM = tf.keras.initializers.glorot_uniform
# INIT_ZEROS = tf.keras.initializers.constant(0.0)
# Activations
ACTIVATION = tf.keras.activations.gelu

VERBOSE = True

In [17]:
FACE_IDXS = [0, 6, 7, 11, 12, 13, 14, 15, 17, 22, 23, 24, 25, 26, 30, 31, 
                     33, 37, 38, 39, 40, 41, 42, 56, 61, 62, 72, 73, 74, 76, 77, 
                     78, 80, 81, 82, 84, 86, 87, 88, 89, 90, 91, 95, 96, 110, 112, 
                     113, 122, 128, 130, 133, 144, 145, 146, 153, 154, 155, 157, 158, 
                     159, 160, 161, 163, 168, 173, 178, 179, 180, 181, 183, 184, 185, 
                     188, 189, 190, 191, 193, 196, 197, 232, 233, 243, 244, 245, 246, 
                     247, 249, 252, 253, 254, 255, 256, 259, 260, 263, 267, 268, 269, 
                     270, 271, 272, 286, 291, 292, 302, 303, 304, 306, 307, 308, 310, 
                     311, 312, 314, 316, 317, 318, 319, 320, 321, 324, 325, 339, 341, 
                     351, 357, 359, 362, 373, 374, 375, 380, 381, 382, 384, 385, 386, 
                     387, 388, 390, 398, 402, 403, 404, 405, 407, 408, 409, 412, 413, 
                     414, 415, 417, 419, 453, 463, 464, 465, 466, 467]
POSE_IDXS = np.arange(489, 514)
LEFT_HAND_IDXS = np.arange(468, 489)
RIGHT_HAND_IDXS = np.arange(522, 543)

# All landmarks that are used for modeling. 
LANDMARK_IDXS = np.concatenate((FACE_IDXS, POSE_IDXS, LEFT_HAND_IDXS, RIGHT_HAND_IDXS))

# Indicies after landmarks have been filtered. 
FACE_START = 0
LEFT_HAND_START = len(FACE_IDXS)
POSE_START = LEFT_HAND_START + len(LEFT_HAND_IDXS)
RIGHT_HAND_START = POSE_START + len(POSE_IDXS)

# Length of landmarks.
FACE_LEN = len(FACE_IDXS)
POSE_LEN = POSE_IDXS.size
LEFT_HAND_LEN = LEFT_HAND_IDXS.size
RIGHT_HAND_LEN = RIGHT_HAND_IDXS.size

In [18]:
class LandmarkEmbedding(tf.keras.layers.Layer):
    def __init__(self, units, name, activation):
        super(LandmarkEmbedding, self).__init__(name=f'{name}_embedding')
        self.UNITS = units
        self.ACTIVATION = activation
        
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'UNITS': self.UNITS,
            'ACTIVATION': self.ACTIVATION,
        })
        return config
        
    def build(self, input_shape):
        # Embedding for missing landmark in frame, initizlied with zeros
        self.empty_embedding = self.add_weight(
            name=f'{self.name}_empty_embedding',
            shape=[self.UNITS],
            initializer=tf.keras.initializers.constant(0.0),
        )
        # Embedding
        self.dense = tf.keras.Sequential([
            tf.keras.layers.Dense(self.UNITS, name=f'{self.name}_dense_1', use_bias=False, 
                                  kernel_initializer=tf.keras.initializers.glorot_uniform, activation=self.ACTIVATION),
            tf.keras.layers.Dense(self.UNITS, name=f'{self.name}_dense_2', use_bias=False, kernel_initializer=tf.keras.initializers.he_uniform),
        ], name=f'{self.name}_dense')

    def call(self, x):
        return tf.where(
                # Checks whether landmark is missing in frame
                tf.reduce_sum(x, axis=2, keepdims=True) == 0,
                # If so, the empty embedding is used
                self.empty_embedding,
                # Otherwise the landmark data is embedded
                self.dense(x),
            )

In [19]:
class Embedding(tf.keras.layers.Layer):
    def __init__(self, input_size, face_units, hands_units, pose_units, units, activation):
        super(Embedding, self).__init__()
        self.INPUT_SIZE = input_size
        self.FACE_UNITS = face_units
        self.HANDS_UNITS = hands_units
        self.POSE_UNITS = pose_units
        self.UNITS = units
        self.ACTIVATION = activation
        
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'INPUT_SIZE': self.INPUT_SIZE,
            'FACE_UNITS': self.FACE_UNITS,
            'HANDS_UNITS': self.HANDS_UNITS,
            'POSE_UNITS': self.POSE_UNITS,
            'UNITS': self.UNITS,
            'ACTIVATION': self.ACTIVATION,
        })
        return config

    def build(self, input_shape):
        # Positional Embedding, initialized with zeros
        self.positional_embedding = tf.keras.layers.Embedding(self.INPUT_SIZE+1, self.UNITS, embeddings_initializer=tf.keras.initializers.constant(0.0))
        # Embedding layer for Landmarks
        self.face_embedding = LandmarkEmbedding(self.FACE_UNITS, 'face', self.ACTIVATION)
        self.left_hand_embedding = LandmarkEmbedding(self.HANDS_UNITS, 'left_hand', self.ACTIVATION)
        self.right_hand_embedding = LandmarkEmbedding(self.HANDS_UNITS, 'right_hand', self.ACTIVATION)
        self.pose_embedding = LandmarkEmbedding(self.POSE_UNITS, 'pose', self.ACTIVATION)
        # Landmark Weights
        self.landmark_weights = tf.Variable(tf.zeros([4], dtype=tf.float32), name='landmark_weights')
        # Fully Connected Layers for combined landmarks
        self.fc = tf.keras.Sequential([
            tf.keras.layers.Dense(self.UNITS, name='fully_connected_1', use_bias=False, 
                                  kernel_initializer=tf.keras.initializers.glorot_uniform, activation=self.ACTIVATION),
            tf.keras.layers.Dense(self.UNITS, name='fully_connected_2', use_bias=False, kernel_initializer=tf.keras.initializers.he_uniform),
        ], name='fc')


    def call(self, face0, left_hand0, right_hand0, pose0, non_empty_frame_idxs,training=False):
        # Face
        face_embedding = self.face_embedding(face0)
        # Left Hand
        left_hand_embedding = self.left_hand_embedding(left_hand0)
        # Right Hand
        right_hand_embedding = self.right_hand_embedding(right_hand0)
        # Pose
        pose_embedding = self.pose_embedding(pose0)
        # Merge Embeddings of all landmarks with mean pooling
        x = tf.stack((face_embedding, left_hand_embedding, right_hand_embedding, pose_embedding), axis=3)
        # Merge Landmarks with trainable attention weights
        x = x * tf.nn.softmax(self.landmark_weights)
        x = tf.reduce_sum(x, axis=3)
        # Fully Connected Layers
        x = self.fc(x)
        # Add Positional Embedding
        normalised_non_empty_frame_idxs = tf.where(
            tf.math.equal(non_empty_frame_idxs, -1.0),
            self.INPUT_SIZE,
            tf.cast(
                non_empty_frame_idxs / tf.reduce_max(non_empty_frame_idxs, axis=1, keepdims=True) * self.INPUT_SIZE,
                tf.int32,
            ),
        )
        x = x + self.positional_embedding(normalised_non_empty_frame_idxs)
        
        return x

In [20]:
# based on: https://stackoverflow.com/questions/67342988/verifying-the-implementation-of-multihead-attention-in-transformer
# replaced softmax with softmax layer to support masked softmax
def scaled_dot_product(q,k,v, softmax, attention_mask):
    #calculates Q . K(transpose)
    qkt = tf.matmul(q,k,transpose_b=True)
    #caculates scaling factor
    dk = tf.math.sqrt(tf.cast(q.shape[-1],dtype=tf.float32))
    scaled_qkt = qkt/dk
    softmax = softmax(scaled_qkt, mask=attention_mask)
    
    z = tf.matmul(softmax,v)
    #shape: (m,Tx,depth), same shape as q,k,v
    return z

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self,d_model,num_of_heads):
        super(MultiHeadAttention,self).__init__()
        self.d_model = d_model
        self.num_of_heads = num_of_heads
        self.depth = d_model//num_of_heads
        self.wq = [tf.keras.layers.Dense(self.depth) for i in range(num_of_heads)]
        self.wk = [tf.keras.layers.Dense(self.depth) for i in range(num_of_heads)]
        self.wv = [tf.keras.layers.Dense(self.depth) for i in range(num_of_heads)]
        self.wo = tf.keras.layers.Dense(d_model)
        self.softmax = tf.keras.layers.Softmax()
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'd_model': self.d_model,
            'num_of_heads': self.num_of_heads
        })
        return config
        
    def call(self,x, attention_mask):
        multi_attn = []
        for i in range(self.num_of_heads):
            Q = self.wq[i](x)
            K = self.wk[i](x)
            V = self.wv[i](x)
            multi_attn.append(scaled_dot_product(Q,K,V, self.softmax, attention_mask))
            
        multi_head = tf.concat(multi_attn,axis=-1)
        multi_head_attention = self.wo(multi_head)
        return multi_head_attention

In [21]:
# Full Transformer
class Transformer(tf.keras.layers.Layer):
    def __init__(self, num_blocks, layer_norm_eps, units, mlp_ratio, mlp_dropout_ratio, activation):
        super(Transformer, self).__init__(name='transformer')
        self.NUM_BLOCKS = num_blocks
        self.LAYER_NORM_EPS = layer_norm_eps
        self.UNITS = units
        self.MLP_RATIO = mlp_ratio
        self.MLP_DROPOUT_RATIO = mlp_dropout_ratio
        self.ACTIVATION = activation
        
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'NUM_BLOCKS': self.NUM_BLOCKS,
            'LAYER_NORM_EPS': self.LAYER_NORM_EPS,
            'MLP_RATIO': self.MLP_RATIO,
            'MLP_DROPOUT_RATIO': self.MLP_DROPOUT_RATIO,
            'UNITS': self.UNITS,
            'ACTIVATION': self.ACTIVATION,
        })
        return config
    
    def build(self, input_shape):
        self.ln_1s = []
        self.mhas = []
        self.ln_2s = []
        self.mlps = []
        # Make Transformer Blocks
        for i in range(self.NUM_BLOCKS):
            # First Layer Normalisation
            self.ln_1s.append(tf.keras.layers.LayerNormalization(epsilon=self.LAYER_NORM_EPS))
            # Multi Head Attention
            self.mhas.append(MultiHeadAttention(self.UNITS, 4))
            # Second Layer Normalisation
            self.ln_2s.append(tf.keras.layers.LayerNormalization(epsilon=self.LAYER_NORM_EPS))
            # Multi Layer Perception
            self.mlps.append(tf.keras.Sequential([
                tf.keras.layers.Dense(self.UNITS * self.MLP_RATIO, activation=self.ACTIVATION, 
                                      kernel_initializer=tf.keras.initializers.glorot_uniform),
                tf.keras.layers.Dropout(self.MLP_DROPOUT_RATIO),
                tf.keras.layers.Dense(self.UNITS, kernel_initializer=tf.keras.initializers.he_uniform),
            ]))
        
    def call(self, x, attention_mask):
        # Iterate input over transformer blocks
        for ln_1, mha, ln_2, mlp in zip(self.ln_1s, self.mhas, self.ln_2s, self.mlps):
            x1 = ln_1(x)
            attention_output = mha(x1, attention_mask)
            x2 = x1 + attention_output
            x3 = ln_2(x2)
            x3 = mlp(x3)
            x = x3 + x2
    
        return x

In [22]:
def get_model():
    # Inputs
    frames = tf.keras.layers.Input([config["INPUT_SIZE"], config["N_COLS"] * config["N_DIMS"]], dtype=tf.float32, name='FRAMES')
    non_empty_frame_idxs = tf.keras.layers.Input([config["INPUT_SIZE"]], dtype=tf.float32, name='NON_EMPTY_FRAME_IDXS')
    
    # Attention Mask
    mask = tf.cast(tf.math.not_equal(non_empty_frame_idxs, -1), tf.float32)
    mask = tf.expand_dims(mask, axis=2)
    
    # Slice out face indicies       
    face = tf.slice(frames, [0, 0, FACE_START], [-1, config["INPUT_SIZE"], FACE_LEN * 2])
    # face = tf.reshape(frames, [-1, config["INPUT_SIZE"], FACE_LEN*2])
    
     # Slice out left_hand indicies
    left_hand = tf.slice(frames, [0, 0, LEFT_HAND_START * 2], [-1, config["INPUT_SIZE"], LEFT_HAND_LEN * 2])
    # left_hand = tf.reshape(frames, [-1, config["INPUT_SIZE"], len(LEFT_HAND_IDXS)*2])

    # Slice out pose indicies
    pose = tf.slice(frames, [0, 0, POSE_START * 2], [-1, config["INPUT_SIZE"], POSE_LEN * 2])
    # pose = tf.reshape(frames, [-1, config["INPUT_SIZE"], len(POSE_IDXS)*2])

    # Slice out right_hand indicies
    right_hand = tf.slice(frames, [0, 0, RIGHT_HAND_START * 2], [-1, config["INPUT_SIZE"], RIGHT_HAND_LEN * 2])
    # right_hand = tf.reshape(frames, [-1, config["INPUT_SIZE"], len(RIGHT_HAND_IDXS)*2])
    
    embedding_layer = Embedding(config["INPUT_SIZE"], FACE_UNITS, HANDS_UNITS, POSE_UNITS, UNITS, ACTIVATION)
    x = embedding_layer(face, left_hand, right_hand, pose, non_empty_frame_idxs)
    transformer_input_shape = x.shape
    
    if (TRANSFORMERV1):
        # Encoder Transformer Blocks
        transformer_layer = Transformer(NUM_BLOCKS, LAYER_NORM_EPS, UNITS, MLP_RATIO, MLP_DROPOUT_RATIO, ACTIVATION)
        x = transformer_layer(x, mask)
    else:
        encoder_input_shape = transformer_input_shape
        for _ in range(NUM_BLOCKS):
            x = TransformerV2(encoder_input_shape, NUM_HEADS, UNITS, MLP_DROPOUT_RATIO, LAYER_NORM_EPS)(x)
            encoder_input_shape = x.shape[1:]  # Update the input shape for the next encoder
    
    # Pooling
    x = tf.reduce_sum(x * mask, axis=1) / tf.reduce_sum(mask, axis=1)
    # Classifier Dropout
    x = tf.keras.layers.Dropout(CLASSIFIER_DROPOUT_RATIO)(x)
    # Classification Layer
    x = tf.keras.layers.Dense(config["NUM_CLASSES"], activation=tf.keras.activations.softmax, kernel_initializer=INIT_GLOROT_UNIFORM)(x)
    
    outputs = x
    
    # Create Tensorflow Model
    model = tf.keras.models.Model(inputs=[frames, non_empty_frame_idxs], outputs=outputs)
    
    # Simple Categorical Crossentropy Loss
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    
    # Adam Optimizer with weight decay
    optimizer = tfa.optimizers.AdamW(learning_rate=config["LEARNING_RATE"], weight_decay=config["WEIGHT_DECAY"])
    
    # TopK Metrics
    metrics = [
        tf.keras.metrics.SparseCategoricalAccuracy(name='acc'),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='top_5_acc'),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10, name='top_10_acc'),
    ]
    
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics, run_eagerly=True)
    
    return model

In [23]:
X = np.load('X.npy')
y = np.load('y.npy')
NON_EMPTY_FRAME_IDXS = np.load('NON_EMPTY_FRAME_IDXS.npy')

In [35]:
# Custom sampler to get a batch containing N times all signs
def get_train_batch(X, y, NON_EMPTY_FRAME_IDXS, batch_size=128):
    # Arrays to store batch in
    X_batch = np.zeros([batch_size, config['INPUT_SIZE'], config['N_COLS'] * config['N_DIMS']], dtype=np.float32)
    y_batch = np.arange(0, batch_size, step=1/batch_size, dtype=np.float32).astype(np.int64)
    non_empty_frame_idxs_batch = np.zeros([batch_size, config['INPUT_SIZE']], dtype=np.float32)
    
    samples = X.shape[0]
    n_batches = samples // batch_size
    
    while True:
        for index in range(n_batches):

            X_batch = X[(index*batch_size):((index+1)*batch_size)]
            y_batch = y[(index*batch_size):((index+1)*batch_size)]
            NON_EMPTY_FRAME_IDXS_batch = NON_EMPTY_FRAME_IDXS[(index*batch_size):((index+1)*batch_size)]

        yield { 'FRAMES': X_batch, 'NON_EMPTY_FRAME_IDXS': NON_EMPTY_FRAME_IDXS_batch }, y_batch

In [25]:
from sklearn.model_selection import train_test_split 

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
NON_EMPTY_FRAME_IDXS_train, NON_EMPTY_FRAME_IDXS_val = train_test_split(NON_EMPTY_FRAME_IDXS, test_size=0.2, random_state=42)

# Learning Rate Scheduler

In [26]:
def lrfn(current_step, num_warmup_steps, lr_max, num_cycles=0.50, num_training_steps=config["N_EPOCHS"], warm_method='log'):
    
    if current_step < num_warmup_steps:
        if warm_method == 'log':
            return lr_max * 0.10 ** (num_warmup_steps - current_step)
        else:
            return lr_max * 2 ** -(num_warmup_steps - current_step)
    else:
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))

        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))) * lr_max

In [27]:
# Learning rate for encoder
LR_SCHEDULE = [lrfn(step, num_warmup_steps=config["N_WARMUP_EPOCHS"], 
                    lr_max=config["LEARNING_RATE"], num_cycles=0.50) for step in range(config["N_EPOCHS"])]

# Learning Rate Callback
lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda step: LR_SCHEDULE[step], verbose=1)

# Weight Decay Callback

In [28]:
# Custom callback to update weight decay with learning rate
class WeightDecayCallback(tf.keras.callbacks.Callback):
    def __init__(self, wd_ratio=config['WD_RATIO']):
        self.step_counter = 0
        self.wd_ratio = wd_ratio
    
    def on_epoch_begin(self, epoch, logs=None):
        model.optimizer.weight_decay = model.optimizer.learning_rate * self.wd_ratio
        print(f'learning rate: {model.optimizer.learning_rate.numpy():.2e}, weight decay: {model.optimizer.weight_decay.numpy():.2e}')

# Training

In [ ]:
# Clear all models in GPU
tf.keras.backend.clear_session()

# Get new fresh model
model = get_model()

# Sanity Check
model.summary()

log_dir = "./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# Actual Training
history = model.fit(
        x=get_train_batch(X_train, y_train, NON_EMPTY_FRAME_IDXS_train, batch_size=config['TRAIN_BATCH_SIZE']),
        steps_per_epoch= len(X_train) // config['TRAIN_BATCH_SIZE'],
        epochs=config["N_EPOCHS"],
        validation_data= ({ 'frames': X_val, 'non_empty_frame_idxs': NON_EMPTY_FRAME_IDXS_val }, y_val),
        batch_size=config["BATCH_SIZE_VAL"],
        callbacks=[
            lr_callback,
            WeightDecayCallback(config["WD_RATIO"]),
#             tensorboard_callback
          ],
        verbose = VERBOSE,
    )

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 NON_EMPTY_FRAME_IDXS (InputLay  [(None, 38)]        0           []                               
 er)                                                                                              
                                                                                                  
 FRAMES (InputLayer)            [(None, 38, 454)]    0           []                               
                                                                                                  
 tf.math.not_equal (TFOpLambda)  (None, 38)          0           ['NON_EMPTY_FRAME_IDXS[0][0]']   
                                                                                                  
 tf.slice (TFOpLambda)          (None, 38, 320)      0           ['FRAMES[0][0]']             

2023-04-19 06:43:17.429632: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


218/590 [==========>...................] - ETA: 1:34 - loss: 86.3063 - acc: 0.0118 - top_5_acc: 0.0652 - top_10_acc: 0.1449

In [ ]:
# Save Model Weights
model.save_weights(f'model.h5')

In [ ]:
# model = get_model()
# model.load_weights('tf_models/model.h5')

# Submission

Submission code loosley based on [this notebook](https://www.kaggle.com/code/dschettler8845/gislr-learn-eda-baseline#baseline) by [Darien Schettler
](https://www.kaggle.com/dschettler8845)

In [ ]:
# TFLite model for submission
class TFLiteModel(tf.keras.Model):
    def __init__(self, model):
        super(TFLiteModel, self).__init__()

        # Load the feature generation and main models
        self.preprocess_layer = preprocess_layer
        self.model = model
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[None, config["N_ROWS"], config["N_DIMS"]], dtype=tf.float32, name='inputs')])
    def call(self, inputs):
        # Preprocess Data
        x, non_empty_frame_idxs = self.preprocess_layer(inputs)
        # Add Batch Dimension
        x = tf.expand_dims(x, axis=0)
        non_empty_frame_idxs = tf.expand_dims(non_empty_frame_idxs, axis=0)
        # Make Prediction
        outputs = self.model({'FRAMES': x, 'NON_EMPTY_FRAME_IDXS': non_empty_frame_idxs })
        # Squeeze Output 1x250 -> 250
        outputs = tf.squeeze(outputs, axis=0)

        # Return a dictionary with the output tensor
        return {'outputs': outputs}

# Define TF Lite Model
tflite_keras_model = TFLiteModel(model)


In [ ]:
# Create Model Converter
keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(tflite_keras_model)
keras_model_converter.experimental_new_converter = True

# Convert Model
tflite_model = keras_model_converter.convert()
# Write Model
with open(f'model.tflite', 'wb') as f:
    f.write(tflite_model)
    
# Zip Model
!zip submission.zip /kaggle/working/model.tflite

In [ ]:
# Sanity Check
demo_raw_data = load_relevant_data_subset(train_metadata['file_path'].values[2])
print(f'demo_raw_data shape: {demo_raw_data.shape}, dtype: {demo_raw_data.dtype}')
demo_output = tflite_keras_model(demo_raw_data)["outputs"]
print(f'demo_output shape: {demo_output.shape}, dtype: {demo_output.dtype}')
demo_prediction = demo_output.numpy().argmax()
print(f'demo_prediction: {demo_prediction}, correct: {train_metadata.iloc[0]["sign_ord"]}')

In [ ]:
# Verify TFLite model can be loaded and used for prediction
# !pip install tflite-runtime
# import tf.lite.interpreter as tflite

interpreter = tf.lite.Interpreter("/kaggle/working/model.tflite")
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn = interpreter.get_signature_runner("serving_default")

output = prediction_fn(inputs=demo_raw_data)
sign = output['outputs'].argmax()

print("PRED : ", ORD2SIGN.get(sign), f'[{sign}]')
print("TRUE : ", train_metadata.sign.values[0], f'[{train_metadata.sign_ord.values[0]}]')